<a href="https://github.com/Deffro/Data-Science-Portfolio/tree/master"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QOuGo78gfqTNq_Dxz6K_nWb08ZLzvukj?usp=sharing)

# **How to do the "Retrieval" in Retrieval-Augmented Generation (RAG)**

## **Introduction**

Efficient and accurate text retrieval is a cornerstone of modern information systems, powering applications like search engines, chatbots, and knowledge bases.

It is the first step in **RAG (Retrieval-Augmented Generation) systems**. RAG systems, first use text retrieval to find the answer to our query and then use an LLM to answer. RAG allows us to "chat with our data".

In this project, we explore the integration of **dense retrieval**, **BM25 lexical search**, and **transformer-based reranking** to create a robust and scalable text retrieval system.

The project leverages the strengths of each technique:
- **Dense Retrieval**: Captures semantic meaning by embedding text into high-dimensional vector spaces, enabling similarity-based search.
- **BM25 Lexical Search**: Performs efficient keyword matching to quickly narrow down relevant results.
- **Transformer-Based Reranking**: Uses Hugging Face cross-encoders to evaluate and rank query-document pairs based on semantic relevance, ensuring precision in the final output.

This hybrid approach optimizes for both computational efficiency and retrieval accuracy, making it well-suited for use cases where context, relevance, and speed are critical.

---

## **Core Concepts and Techniques**

1. **Chunking and Embedding**:
   - Text is segmented into chunks (e.g., sentences or paragraphs) to ensure embeddings represent actionable parts of the content.
   - Multiple chunking strategies are explored, including **fixed-length chunks** and **overlapping chunks**, to preserve context and relevance.
   - Embeddings are generated using `SentenceTransformers` to map each chunk into a dense vector space.

2. **Dense Retrieval with FAISS**:
   - FAISS, a library for efficient similarity search, is used to index and retrieve text embeddings.
   - Queries are embedded into the same vector space, and FAISS is queried to return the nearest neighbors.

3. **BM25 Lexical Search**:
   - BM25 scores documents based on keyword overlap with the query, serving as an efficient initial filter for relevant documents.
   - Top candidates from BM25 are passed to the reranking stage for further refinement.

4. **Reranking with Cross-Encoders**:
   - A transformer-based cross-encoder evaluates query-document pairs, capturing nuanced semantic relationships.
   - Candidates are reranked based on relevance scores, ensuring that the most contextually accurate results are returned.

5. **Flexible Chunking Strategies**:
   - **Fixed-Length Chunks**: Group text into fixed-length units (e.g., 3 sentences per chunk) to balance granularity and computational efficiency.
   - **Overlapping Chunks**: Include overlapping sentences between chunks to preserve contextual continuity and avoid information loss.

---

## **Applications**

1. **Document Retrieval**:
   - Efficiently retrieve relevant sections from large text corpora, such as research papers, books, or legal documents.
   
2. **Conversational AI**:
   - Enhance chatbot responses by retrieving the most contextually relevant knowledge base entries.

3. **Content Recommendation**:
   - Suggest similar articles, reports, or documents based on user queries.

4. **Enterprise Search**:
   - Facilitate internal search systems for knowledge management and decision-making.


In [1]:
%%capture
!pip install faiss-gpu==1.7.2 rank_bm25==0.2.2 sentence-transformers==3.0.1

# **Dense Retrieval**

Dense retrieval uses embeddings to represent text as numeric vectors. These vectors are positioned in a high-dimensional space, where similar texts are close to each other.

- **Embeddings**: Numeric representations of text, capturing semantic meaning.
- **Vector Space**: A mathematical space where texts are points, and their proximity represents similarity.

#### **How It Works**
1. Embed both the query and the document corpus into a shared vector space.
2. Compute the similarity between the query embedding and the document embeddings (e.g., using cosine similarity or Euclidean distance).
3. Return the nearest neighbors as the search results.


## **The Process**

### 1. Chunking the Text
Text is divided into manageable chunks (e.g., sentences or paragraphs) to ensure embeddings can effectively represent specific, actionable parts of the text.

### 2. Embedding the Text Chunks
We can use `SentenceTransformers` for that.

### 3. Building the Search Index
Once embeddings are created, they are stored in a FAISS index for efficient nearest neighbor search.

### 4. Searching the Index
The query is embedded, and the index is queried for the nearest neighbors.

## **Advanced Topics**

### 1. Filtering Results
Set a threshold for similarity scores to filter out irrelevant results:

### 2. Document-Level Search
Instead of splitting into sentences, larger chunks (e.g., paragraphs) can be used for broader context.

### 3. Combining Methods
Use dense retrieval for initial results, followed by reranking with a more sophisticated model.

## **Applications**
1. Document Retrieval: Find relevant sections in large text archives.
2. Chatbots: Retrieve answers from a knowledge base for conversational agents.
3. Content Recommendation: Suggest similar articles or documents.

---


Let's start by loading the text for the popular ancient Greek epic poem "Odyssey".

In [3]:
text = """
The Odyssey is one of two major ancient Greek epic poems attributed to Homer.
It is one of the oldest works of literature still widely read by modern audiences.
Like the Iliad, the Odyssey is divided into 24 books.

The Odyssey begins after the end of the ten-year Trojan War (the subject of the Iliad), from which Odysseus (also known by the Latin variant Ulysses), king of Ithaca, has still not returned because he angered Poseidon, the god of the sea.
Odysseus's son, Telemachus, is about 20 years old and is sharing his absent father's house on the island of Ithaca with his mother Penelope and the suitors of Penelope, a crowd of 108 boisterous young men who each aim to persuade Penelope for her hand in marriage, all the while reveling in the king's palace and eating up his wealth.

Odysseus's protectress, the goddess Athena, asks Zeus, king of the gods, to finally allow Odysseus to return home when Poseidon is absent from Mount Olympus.
Disguised as a chieftain named Mentes, Athena visits Telemachus to urge him to search for news of his father.
He offers her hospitality, and they observe the suitors dining rowdily while Phemius, the bard, performs a narrative poem for them.

That night, Athena, disguised as Telemachus, finds a ship and crew for the true prince.
The next morning, Telemachus calls an assembly of citizens of Ithaca to discuss what should be done with the insolent suitors, who then scoff at Telemachus.
Accompanied by Athena (now disguised as Mentor), the son of Odysseus departs for the Greek mainland to the household of Nestor, most venerable of the Greek warriors at Troy, who resided in Pylos after the war.

From there, Telemachus rides to Sparta, accompanied by Nestor's son.
There he finds Menelaus and Helen, who are now reconciled.
Both Helen and Menelaus also say that they returned to Sparta after a long voyage by way of Egypt.
There, on the island of Pharos, Menelaus encounters the old sea-god Proteus, who tells him that Odysseus was a captive of the nymph Calypso.
Telemachus learns the fate of Menelaus's brother, Agamemnon, king of Mycenae and leader of the Greeks at Troy: he was murdered on his return home by his wife Clytemnestra and her lover Aegisthus.
The story briefly shifts to the suitors, who have only just realized that Telemachus is gone.
Angry, they formulate a plan to ambush his ship and kill him as he sails back home. Penelope overhears their plot and worries for her son's safety."""

# Split the text into sentences
sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]


We splitted the text into sentences, so each sentence will be a chunk. A chunk is what a retrieval system can return as an answer to a query.

### Embedding the text chunks**

Next, we convert each sentence (chunk) into a numerical representation called an embedding.

**What Are Embeddings?**

Embeddings are numerical representations of text, typically in a fixed-dimensional vector space. These representations allow us to measure semantic similarity between chunks of text.

- We use the `all-mpnet-base-v2` model, which is optimized for creating embeddings that capture semantic meaning.
- The encode method converts each sentence into a 768-dimensional embedding vector.

In [4]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to embeddings
embeds = model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
print(embeds.shape)

(19, 768)


We can see that 19 embeddings (our 19 sentences) with a dimension of 768 each were created.

768 is the embedding dimensions of the `all-mpnet-base-v2` model we used from [Hugging Face](https://huggingface.co/sentence-transformers/all-mpnet-base-v2).

### Building The Search Index

To enable fast and efficient retrieval, we store the embeddings in a vector database.

For this, we use **FAISS (Facebook AI Similarity Search)**, a library designed for high-speed similarity searches.

In [7]:
import numpy as np
import pandas as pd
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

- `IndexFlatL2`: This creates an index that uses the L2 (Euclidean) distance metric to measure similarity between vectors.
- `add`: Adds the embeddings to the index, making them searchable.

**Why Use FAISS?**

FAISS is optimized for:

- Scalability: Handles millions of embeddings efficiently.
- Speed: Performs nearest-neighbor searches in milliseconds.

Pro Tip: Convert embeddings to 32-bit floating-point numbers (np.float32) before adding them to the index. This ensures compatibility with FAISS.

### Querying the FAISS Index

Now that we have built our FAISS index, the next step is to query it and retrieve the most relevant text chunks for a given query.

---

**Search the Index**

Using the FAISS index, we can retrieve the nearest neighbors (most similar chunks) to our query. Each query is embedded into the same vector space as the indexed chunks, and FAISS finds the closest matches.


In [8]:
def retrieve(query, number_of_results=3):
    """
    Search for the nearest neighbors of a query in the FAISS index.

    Parameters:
    - query (str): The query text to search for.
    - number_of_results (int): Number of nearest neighbors to retrieve.

    Returns:
    - pd.DataFrame: A DataFrame containing the nearest texts and their distances.
    """
    # 1. Get the query's embedding
    query_embed = model.encode([query])  # Ensure query is encoded as a list

    # 2. Retrieve the nearest neighbors
    distances, similar_item_ids = index.search(np.float32(query_embed), number_of_results)

    # 3. Format the results
    texts_np = np.array(texts)  # Convert texts list to numpy array for indexing
    results = pd.DataFrame(data={
        'texts': texts_np[similar_item_ids[0]],
        'distance': distances[0]
    })

    return results

**1. Embedding the Query:**
- The query is passed through the same model used for embedding the text chunks. This ensures that the query vector is in the same semantic space as the indexed vectors.

**2. Finding Nearest Neighbors:**
- The FAISS index is queried using the embedded vector.
- `index.search` retrieves the `number_of_results` nearest neighbors, returning their indices and distances.

**3. Formatting the Results:**
- The indices are used to fetch the corresponding text chunks.
- The results are stored in a DataFrame, which includes the retrieved texts and their distances.


In [9]:
query = "which character had the first encounter with a god"
results = retrieve(query)
results

,texts,distance
0,The Odyssey begins after the end of the ten-ye...,1.302391
1,"There he finds Menelaus and Helen, who are now...",1.399046
2,"Disguised as a chieftain named Mentes, Athena ...",1.415817


In the above example, each sentence is a chunk and thus a sentence can be returned as a query result.

In [10]:
results["texts"][0]

"The Odyssey begins after the end of the ten-year Trojan War (the subject of the Iliad), from which Odysseus (also known by the Latin variant Ulysses), king of Ithaca, has still not returned because he angered Poseidon, the god of the sea Odysseus's son, Telemachus, is about 20 years old and is sharing his absent father's house on the island of Ithaca with his mother Penelope and the suitors of Penelope, a crowd of 108 boisterous young men who each aim to persuade Penelope for her hand in marriage, all the while reveling in the king's palace and eating up his wealth Odysseus's protectress, the goddess Athena, asks Zeus, king of the gods, to finally allow Odysseus to return home when Poseidon is absent from Mount Olympus"

### Multiple sentences as a chunk

In this approach, text is divided into chunks containing a fixed number of consecutive sentences. Each chunk represents a self-contained segment of text, making it easier to process and retrieve relevant results.

### **Advantages**
- **Compact Representation**: Larger chunks reduce the number of embeddings required, making the system more efficient.
- **Context Preservation**: Grouping multiple sentences together captures a more holistic view of the content, ensuring that related ideas are stored together.
- **Flexibility**: The number of sentences per chunk can be adjusted to suit the use case (e.g., larger chunks for narratives, smaller chunks for technical data).


In [11]:
def chunk_sentences(sentences, sentences_per_chunk=3):
    """
    Chunk sentences into groups of a specified number.

    Parameters:
    - sentences (list): A list of sentences.
    - sentences_per_chunk (int): The number of sentences per chunk.

    Returns:
    - list: A list of chunks where each chunk contains the specified number of sentences.
    """
    chunks = [
        ' '.join(sentences[i:i + sentences_per_chunk])
        for i in range(0, len(sentences), sentences_per_chunk)
    ]
    return chunks

# Specify the number of sentences per chunk
sentences_per_chunk = 3
texts = chunk_sentences(sentences, sentences_per_chunk)

# Load SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert chunks to embeddings
embeds = model.encode(texts, show_progress_bar=True)

# Initialize FAISS index
dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

query = "which character had the first encounter with a god"
results = retrieve(query)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,texts,distance
0,The Odyssey begins after the end of the ten-ye...,1.302391
1,"There he finds Menelaus and Helen, who are now...",1.399046
2,"Disguised as a chieftain named Mentes, Athena ...",1.415817


In [12]:
results["texts"][0]

"The Odyssey begins after the end of the ten-year Trojan War (the subject of the Iliad), from which Odysseus (also known by the Latin variant Ulysses), king of Ithaca, has still not returned because he angered Poseidon, the god of the sea Odysseus's son, Telemachus, is about 20 years old and is sharing his absent father's house on the island of Ithaca with his mother Penelope and the suitors of Penelope, a crowd of 108 boisterous young men who each aim to persuade Penelope for her hand in marriage, all the while reveling in the king's palace and eating up his wealth Odysseus's protectress, the goddess Athena, asks Zeus, king of the gods, to finally allow Odysseus to return home when Poseidon is absent from Mount Olympus"

### Overlapping Chunks

Overlapping chunks include some sentences from the surrounding text to provide additional context. This approach ensures that important information at the edges of chunks is not lost, which can be critical when adjacent chunks share related content.

### **How It Works**
Each chunk contains a defined number of sentences, but subsequent chunks overlap by a specified number of sentences. For example, with a chunk size of 3 sentences and an overlap of 1 sentence:

- **Chunk 1**: Sentences 1, 2, 3
- **Chunk 2**: Sentences 3, 4, 5
- **Chunk 3**: Sentences 5, 6, 7

### **Advantages**
- **Improved Context**: By including neighboring sentences, overlapping chunks preserve more context, which is especially useful for text with interdependent ideas.
- **Reduced Fragmentation**: Important information at the boundaries of chunks is retained, minimizing the loss of meaning during search.
- **Higher Relevance**: The overlapping ensures that relevant content is less likely to be excluded when a query is processed.


In [13]:
def chunk_sentences_with_context(sentences, sentences_per_chunk=3, overlap=1):
    """
    Chunk sentences into groups with overlap for context.

    Parameters:
    - sentences (list): A list of sentences.
    - sentences_per_chunk (int): The number of sentences per chunk.
    - overlap (int): The number of sentences that should overlap between chunks.

    Returns:
    - list: A list of overlapping chunks.
    """
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk - overlap):
        chunk = sentences[i:i + sentences_per_chunk]
        chunks.append(' '.join(chunk))
        # Break if we've reached the end of the list
        if i + sentences_per_chunk >= len(sentences):
            break
    return chunks

# Specify the number of sentences per chunk and overlap
sentences_per_chunk = 3
overlap = 1
texts = chunk_sentences_with_context(sentences, sentences_per_chunk, overlap)

# Load SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert chunks to embeddings
embeds = model.encode(texts, show_progress_bar=True)

# Initialize FAISS index
dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

query = "which character had the first encounter with a god"
results = retrieve(query)
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,texts,distance
0,"There, on the island of Pharos, Menelaus encou...",1.394832
1,"There he finds Menelaus and Helen, who are now...",1.399046
2,"Disguised as a chieftain named Mentes, Athena ...",1.415817


In [14]:
results["texts"][0]

"There, on the island of Pharos, Menelaus encounters the old sea-god Proteus, who tells him that Odysseus was a captive of the nymph Calypso Telemachus learns the fate of Menelaus's brother, Agamemnon, king of Mycenae and leader of the Greeks at Troy: he was murdered on his return home by his wife Clytemnestra and her lover Aegisthus The story briefly shifts to the suitors, who have only just realized that Telemachus is gone"

# **BM25 and Reranking with Hugging Face Cross-Encoder**

The goal is to enhance search precision by leveraging BM25 for initial candidate selection and using a transformer-based model for reranking those candidates based on semantic similarity.

### **1. BM25 (Lexical Search)**
BM25 is a popular information retrieval algorithm that ranks documents based on keyword relevance. It uses term frequency and inverse document frequency (TF-IDF) to compute scores. BM25 excels in finding documents with overlapping terms from the query.

#### **Key Features of BM25**:
- **Exact Keyword Matching**: Scores are based on the presence and frequency of query terms in the document.
- **Efficient Candidate Retrieval**: Quickly narrows down the search space to a manageable set of top candidates.

#### **Limitations**:
- BM25 cannot capture semantic similarity. For example, it may not identify synonyms or contextually related terms.

---

### **2. Cross-Encoder for Reranking**
A cross-encoder is a transformer model that evaluates the relevance of a query-document pair. It processes both the query and a document together and outputs a relevance score.

#### **Key Features**:
- **Semantic Understanding**: Models can capture the meaning of words and sentences, making them robust to variations in phrasing.
- **Context-Aware**: Each query-document pair is evaluated as a single input, allowing for better contextual understanding.

#### **Advantages of Combining BM25 and Cross-Encoder**:
1. **Efficiency**: BM25 reduces the search space to a smaller set of candidates.
2. **Precision**: Cross-encoder reranking ensures that the most relevant results are returned based on semantic similarity.


In [15]:
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import torch

def bm25_and_rerank(texts, query, top_k=3, num_candidates=10):
    """
    Combine BM25 search with Hugging Face cross-encoder reranking.

    Parameters:
    - texts (list): List of texts to search through.
    - query (str): Search query.
    - top_k (int): Number of top results to return after reranking.
    - num_candidates (int): Number of candidates to retrieve from BM25 for reranking.

    Returns:
    - List of tuples containing document and relevance score.
    """
    print(f"Query: {query}")

    # Preprocess texts for BM25
    tokenized_corpus = [text.split() for text in texts]
    bm25 = BM25Okapi(tokenized_corpus)

    # BM25 search
    tokenized_query = query.split()
    bm25_scores = bm25.get_scores(tokenized_query)

    # Ensure num_candidates does not exceed the number of documents
    num_candidates = min(num_candidates, len(texts))

    top_n_indices = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n_indices]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print("\nTop BM25 candidates:")
    candidates = []
    for hit in bm25_hits[:num_candidates]:
        print(f"BM25 Score: {hit['score']:.3f}, Text: {texts[hit['corpus_id']]}")
        candidates.append(texts[hit['corpus_id']])

    # Rerank with Hugging Face cross-encoder
    inputs = [(query, doc) for doc in candidates]
    encodings = tokenizer.batch_encode_plus(
        inputs,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    encodings = {key: val.to(device) for key, val in encodings.items()}
    with torch.no_grad():
        rerank_scores = model(**encodings).logits.squeeze(-1).cpu().numpy()

    # Combine scores and rerank
    reranked_results = sorted(zip(rerank_scores, candidates), key=lambda x: x[0], reverse=True)

    print("\nTop reranked results:")
    for idx, (score, doc) in enumerate(reranked_results[:top_k]):
        print(f"{idx + 1}. Relevance Score: {score:.3f}, Text: {doc}")

    return reranked_results[:top_k]


# Specify the number of sentences per chunk and overlap
sentences_per_chunk = 3
overlap = 1
texts = chunk_sentences_with_context(sentences, sentences_per_chunk, overlap)

# Load Hugging Face model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-TinyBERT-L-2")
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-TinyBERT-L-2")

query = "which character had the first encounter with a god"

# Call the function
reranked_results = bm25_and_rerank(texts, query, top_k=3, num_candidates=10)

tokenizer_config.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

Query: which character had the first encounter with a god

Top BM25 candidates:
BM25 Score: 4.032, Text: Like the Iliad, the Odyssey is divided into 24 books The Odyssey begins after the end of the ten-year Trojan War (the subject of the Iliad), from which Odysseus (also known by the Latin variant Ulysses), king of Ithaca, has still not returned because he angered Poseidon, the god of the sea Odysseus's son, Telemachus, is about 20 years old and is sharing his absent father's house on the island of Ithaca with his mother Penelope and the suitors of Penelope, a crowd of 108 boisterous young men who each aim to persuade Penelope for her hand in marriage, all the while reveling in the king's palace and eating up his wealth
BM25 Score: 1.475, Text: That night, Athena, disguised as Telemachus, finds a ship and crew for the true prince The next morning, Telemachus calls an assembly of citizens of Ithaca to discuss what should be done with the insolent suitors, who then scoff at Telemachus Ac

After running the query, the system first finds the top 10 chunks based on keyword relevance (using BM25) and then outputs the top 3 most relevant chunks based on the cross-encoder ranking.